## Keypoint Detection Solution
---

In [22]:
# load packages
import cv2
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re
%matplotlib inline
print("Done!")

Done!


In [23]:
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create(30)
surf = cv2.xfeatures2d.SURF_create(300)
print("Done!")

Done!


In [24]:
def getPatches(kps, img, size=32, num=500):
    res = torch.zeros(num, 1, size, size)
    if type(img) is np.ndarray:
        img = torch.from_numpy(img)
    h, w = img.shape      # note: for image, the x direction is the verticle, y-direction is the horizontal...
    for i in range(num):
        cx, cy = kps[i]
        cx, cy = int(cx), int(cy)
        dd = int(size/2)
        xmin, xmax = max(0, cx - dd), min(w, cx + dd ) - 1
        ymin, ymax = max(0, cy - dd), min(h, cy + dd ) - 1 
        
        xmin_res, xmax_res = dd - min(dd,cx), dd + min(dd, w - cx)-1
        ymin_res, ymax_res = dd - min(dd,cy), dd + min(dd, h - cy)-1

        res[i, 0, ymin_res: ymax_res, xmin_res: xmax_res] = img[ymin: ymax, xmin: xmax]
    return res
print("Done!")

Done!


In [26]:
#img_dir = "../image_retrieval/images"
img_dir = "../image_retrieval/query"
#img_dir = "../image_retrieval/all"
kps_num = 30
patch_size = 32
#number_of_images = 140
number_of_images = 35
#number_of_images = 175
res = torch.zeros(number_of_images, kps_num, 2)
patches = torch.zeros(number_of_images, kps_num, 1, patch_size, patch_size)
if os.path.exists(img_dir):
    if os.listdir(img_dir) is []:
        print("No images!")
        exit(0)
    num_img = len(os.listdir(img_dir))
    idx = 0
    sorted_listdir = sorted(os.listdir(img_dir), key=lambda e: [int(s) for s in re.split('[q.]', e) if s.isdigit()])
    for img in sorted_listdir:
        print(img)
        if not img.endswith("JPG"):
            continue
        image_dir = os.path.join(img_dir, img)
        image = cv2.imread(image_dir)
        img= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
        ## find the keypoints and descriptors with SIFT
        if (image_dir != "../image_retrieval/all\q17.JPG" and image_dir != "../image_retrieval/query\q17.JPG"):
            kps, des = sift.detectAndCompute(img, None)
            keypoints_img = [kps[a].pt for a in range(kps_num)] 
            res[idx] = torch.FloatTensor(keypoints_img)
        else:
            kps, des = surf.detectAndCompute(img, None)
            keypoints_img = [kps[a].pt for a in range(kps_num)] 
            res[idx] = torch.FloatTensor(keypoints_img)
        ## extract patches
        patches[idx] = getPatches(keypoints_img, img, size=patch_size, num=kps_num)
        idx += 1
else:
    print("image folder not exists!")
    exit(0)
print("Done!")

q1.JPG
q2.JPG
q3.JPG
q4.JPG
q5.JPG
q6.JPG
q7.JPG
q8.JPG
q9.JPG
q10.JPG
q11.JPG
q12.JPG
q13.JPG
q14.JPG
q15.JPG
q16.JPG
q17.JPG
q18.JPG
q19.JPG
q20.JPG
q21.JPG
q22.JPG
q23.JPG
q24.JPG
q25.JPG
q26.JPG
q27.JPG
q28.JPG
q29.JPG
q30.JPG
q31.JPG
q32.JPG
q33.JPG
q34.JPG
q35.JPG
Done!


In [27]:
print(res.shape)
print(patches.shape)
print("Done!")

torch.Size([35, 30, 2])
torch.Size([35, 30, 1, 32, 32])
Done!


In [28]:
## save tensors
#output_dir_kps = "keypoints-i.pt"
#output_dir_patches = "patches-i.pt"
output_dir_kps = "keypoints-q.pt"
output_dir_patches = "patches-q.pt"
#output_dir_kps = "keypoints-a.pt"
#output_dir_patches = "patches-a.pt"
torch.save(res, output_dir_kps)
torch.save(patches, output_dir_patches)
print("Done!")

Done!
